In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions

configuration removed

In [0]:
df_ath = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://bronze-tokyo@tokyoolympicsdl.dfs.core.windows.net/Athletes")

In [0]:
df_Coaches = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://bronze-tokyo@tokyoolympicsdl.dfs.core.windows.net/Coaches")
df_EntriesGender = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://bronze-tokyo@tokyoolympicsdl.dfs.core.windows.net/EntriesGender")
df_Medals = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://bronze-tokyo@tokyoolympicsdl.dfs.core.windows.net/Medals")
df_Teams = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("abfss://bronze-tokyo@tokyoolympicsdl.dfs.core.windows.net/Teams")

In [0]:
df_EntriesGender = df_EntriesGender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
    .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
df_EntriesGender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
df_Medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- TeamCountry: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [0]:
top_gold_medal_countries = df_Medals.orderBy("Gold", ascending=False).select("TeamCountry","Gold").show()

+--------------------+----+
|         TeamCountry|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = df_EntriesGender.withColumn(
    'Avg_Female', df_EntriesGender['Female'] / df_EntriesGender['Total']
).withColumn(
    'Avg_Male', df_EntriesGender['Male'] / df_EntriesGender['Total']
)
average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
df_ath.write.format("parquet").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/Parquet/Athletes").save()
df_Coaches.write.format("parquet").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/Parquet/Coaches").save()
df_EntriesGender.write.format("parquet").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/Parquet/EntriesGender").save()
df_Medals.write.format("parquet").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/Parquet/Medals").save()
df_Teams.write.format("parquet").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/Parquet/Teams").save()

In [0]:
df_ath.write.format("csv").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/csv/Athletes").save()
df_Coaches.write.format("csv").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/csv/Coaches").save()
df_EntriesGender.write.format("csv").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/csv/EntriesGender").save()
df_Medals.write.format("csv").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/csv/Medals").save()
df_Teams.write.format("csv").mode("append").option("path", "abfss://silver-tokyo@tokyoolympicsdl.dfs.core.windows.net/csv/Teams").save()